In [1]:
import sys
#!{sys.executable} -m pip install selenium
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install numpy

In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np
from time import sleep

In [3]:
#open a Chrome session
driver = webdriver.Chrome('/Applications/chromedriver')

In [ ]:
#open all of Christie's jewelry archives
driver.get('https://www.christies.com/departments/Jewellery-33-1.aspx?pagesection=results#results')

#SOMETIMES THERE IS POP UP THAT NEEDS TO B CLOSED, DEAL WITH THAT
#sleep(10)
#close_pop_up = driver.find_element_by_id('close_signup')
#close_pop_up.click()

view_all = driver.find_element_by_id('divViewAll')
view_all.click()

In [ ]:
#lists of auction links, auction names, and auction dates and locations
urls = []
names = []
dates_locs = []

auction_results = driver.find_elements_by_id('Calendar-Results-Sales')


for auction in auction_results:
    urls.append(auction.find_element_by_css_selector('a').get_attribute('href'))
    dates_locs.append(auction.find_element_by_class_name('col-md-2').text)
    names.append(auction.find_element_by_class_name('image-description--title').text)

In [ ]:
#create dictionary with above lists
auctions = {key: (v1, v2) for key, v1, v2 in zip(urls, dates_locs, names)}

In [ ]:
print(len(auctions))

In [ ]:
# there are many failed scraps happening for the first 20 auctions: check nothing is going wrong

In [ ]:
#a few auction webpages do not follow the main general template, and will not successfully get scraped
#let's record these
failed_scrap = []

#inialize empty dictionary. will contain: key - lot name and descrip, value - price estimates and price realized.
dic = {}



#loop through auction_urls 
for auction in auctions:
    
    auc_name = auctions[auction[0]]
    auc_date_loc = auctions[auction[1]]
    
    
    driver.get(auction)
    sleep(10)
    
    

    try:
        load_all = driver.find_element_by_id("loadAllUpcomingPast")
        load_all.click()
        sleep(20)
        
        #list of links for every lot in current auction
        lots = driver.find_elements_by_class_name('image-preview-container')
        lot_urls = []
        for lot in lots:
            lot_urls.append(lot.find_element_by_css_selector('a').get_attribute('href')) 
            
        #fill dic    
        for lot_url in lot_urls:
            try:
                driver.get(lot_url)
                descrip = driver.find_element_by_id("main_center_0_lblLotDescription")
                key = descrip.text
                price_realized = driver.find_element_by_id('main_center_0_lblPriceRealizedPrimary')
                dic[key] = [price_realized.text]
                price_estimated = driver.find_element_by_id('main_center_0_lblPriceEstimatedPrimary')
                dic[key].append(price_estimated.text)
                #need to add auction date_loc and name
                
                dic[key].append(auc_name)
                dic[key].append(auc_date_loc)
            except:
                continue
    except:
        #record failed scrapes
        failed_scrap.append(auction_url)
        continue
    

In [ ]:
print(failed_scrap)

In [ ]:
print(len(dic))


In [ ]:
final_dict = dic[key]

In [ ]:
#create pandas dataframe from dic
df = pd.DataFrame.from_dict(dic, orient='index', columns = ['price_realized', 'price_estimated'])

In [ ]:
df = df.reset_index()
df = df.rename(columns={"index": 'name/decrip'})
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv(r'/Users/katieross/Desktop/auction_datasets/test2.csv')